In [1]:
import pandas as pd
import os
import sys
import scanpy as sc
import scipy.sparse as sp
import time

In [2]:
fpath = "/scratch/indikar_root/indikar1/shared_data/chondroform/single_cell_data/fan_2024/sc_counts.txt"

df = pd.read_csv(fpath, sep="\t", nrows=10)
print(f"{df.shape=}")
gb = df.memory_usage().sum() / 1e9
print(f"Used {gb}Gb")
df.head()

df.shape=(10, 135896)
Used 0.01087176Gb


,S1WK.AAACCCACATCAACCA.1,S1WK.AAACCCACATGTTCAG.1,S1WK.AAACCCAGTCAGGTGA.1,S1WK.AAACCCAGTCTGCAAT.1,S1WK.AAACCCATCAGACTGT.1,S1WK.AAACCCATCTGCGGAC.1,S1WK.AAACGAACAACTGCTA.1,S1WK.AAACGAACAACTTGGT.1,S1WK.AAACGAACAAGAATGT.1,S1WK.AAACGAACACTGCGTG.1,...,M61RNK.TTTGTTGAGATTGCGG.1,M61RNK.TTTGTTGAGTAACCTC.1,M61RNK.TTTGTTGAGTCCTACA.1,M61RNK.TTTGTTGCAAGTACCT.1,M61RNK.TTTGTTGCACCGGAAA.1,M61RNK.TTTGTTGCACCGTGAC.1,M61RNK.TTTGTTGCATGACAAA.1,M61RNK.TTTGTTGGTACAAGTA.1,M61RNK.TTTGTTGTCTCTATGT.1,M61RNK.TTTGTTGTCTTAGGAC.1
AL627309.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC01409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC01128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC00115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
FAM41C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
start_time = time.time()

adata = sc.read_text(
    fpath,
    delimiter='\t',
    first_column_names=True,
)

end_time = time.time()
elapsed_seconds = end_time - start_time
elapsed_minutes = elapsed_seconds / 60

print(f"Time to read data: {elapsed_minutes:.2f} minutes")

sc.logging.print_memory_usage()

adata

Time to read data: 12.61 minutes
Memory usage: current 23.78 GB, difference +23.78 GB


AnnData object with n_obs × n_vars = 38680 × 135896

In [9]:
adata.X = sp.csr_matrix(adata.X)
sc.logging.print_memory_usage()

Memory usage: current 6.15 GB, difference -17.63 GB


In [10]:
outpath = "/scratch/indikar_root/indikar1/shared_data/chondroform/single_cell_data/fan_2024/sc_counts.h5ad"
adata.write(outpath)
adata


AnnData object with n_obs × n_vars = 38680 × 135896

In [11]:
start_time = time.time()

bdata = sc.read_h5ad(outpath)
sc.logging.print_memory_usage()

end_time = time.time()
elapsed_seconds = end_time - start_time
elapsed_minutes = elapsed_seconds / 60

print(f"Time to read data: {elapsed_minutes:.2f} minutes")

sc.logging.print_memory_usage()

bdata

Memory usage: current 7.12 GB, difference +0.97 GB
Time to read data: 0.05 minutes
Memory usage: current 7.12 GB, difference +0.00 GB


AnnData object with n_obs × n_vars = 38680 × 135896

In [7]:
def read_sparse_matrix(filename):
    """Reads a sparse matrix from a tab-separated file with headers and index names.

    Assumes the file format is:
    header1\theader2\theader3 ...
    index1\trow\tcol\tvalue
    index2\trow\tcol\tvalue
    ...

    Returns:
        tuple: (headers, index_names, sparse_matrix)
    """
    rows = []
    cols = []
    data = []
    headers = []
    index_names = []

    with open(filename, 'r') as file:
        first_line = file.readline().strip()
        headers = first_line.split('\t')  # Split by tabs

        for line in file:
            index, r, c, v = line.strip().split('\t')  # Split by tabs
            index_names.append(index)
            rows.append(int(r))
            cols.append(int(c))
            data.append(int(v))

    matrix = sp.csc_matrix((data, (rows, cols)))
    return headers, index_names, matrix


start_time = time.time()
headers, index_names, matrix = read_sparse_matrix(fpath)
end_time = time.time()
elapsed_seconds = end_time - start_time
elapsed_minutes = elapsed_seconds / 60

print(f"Time to read data: {elapsed_minutes:.2f} minutes")
print("Headers: ", len(headers))
print("Index names: ", len(index_names))
print("Sparse Matrix: ", matrix.shape)

ValueError: too many values to unpack (expected 4)